TO DO:
- Add option to force players into the team, i.e., optimize given certain player are already locked in

In [1]:
# previous gameweek
gameweek=20

In [2]:
import pandas as pd
import numpy as np
import json
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [3]:
# fetch player_dict 
with open('../data/player_name_dict.txt', 'r') as f:
    player_name_dict_as_text = f.read()
    player_name_dict = json.loads(player_name_dict_as_text)

# Solver definition

In [4]:
# function to solve the optimization problem
def optimize(df, max_price, expected_column):
    
    # PRELIMINARIES

    # Create the model
    model = LpProblem(name="FPL", sense=LpMaximize)    
    variables = [LpVariable(name=f'{ix}', cat='Binary') for ix in df.index]
    captaincy_variables = [LpVariable(name=f'cap_{ix}', cat='Binary') for ix in df.index]
    prices = [df.loc[ix,'price'] for ix in df.index]
    # measure of player quality
    expected_points = [df.loc[ix,expected_column] for ix in df.index]
    goalkeepers = [1.0 if df.loc[ix,'position']=='GK' else 0.0 for ix in df.index]
    defenders = [1.0 if df.loc[ix,'position']=='DEF' else 0.0 for ix in df.index]
    midfielders = [1.0 if df.loc[ix,'position']=='MID' else 0.0 for ix in df.index]
    forwards = [1.0 if df.loc[ix,'position']=='FWD' else 0.0 for ix in df.index]
    teams = []
    for team in df['team'].unique():
        teams.append( [1.0 if df.loc[ix,'team']==team else 0.0 for ix in df.index] )

    # CONSTRAINTS

    # select 11 players
    model += lpSum(variables) == 11
    # select 1 captain
    model += lpSum(captaincy_variables) == 1
    # captain must be one of the 11 players in the team
    for i in range(0,len(variables)):
        model += captaincy_variables[i] <= variables[i]
    # set maximum price for starting 11
    model += np.dot(prices,variables) <= max_price
    # only 1 goalkeeper
    model += np.dot(goalkeepers,variables) == 1
    # at least 3 defenders
    model += np.dot(defenders,variables) >= 3
    # at most 5 defenders
    model += np.dot(defenders,variables) <= 5
    # at most 5 midfielders
    model += np.dot(midfielders,variables) <= 5
    # at least 1 forward
    model += np.dot(forwards,variables) >= 1
    # at most 3 forwards
    model += np.dot(forwards,variables) <= 3
    # max 3 players from any given team
    for team in teams:
        model += np.dot(team,variables) <= 3

    # OBJECTIVE
    # the second part doubles the captain's points
    model += np.dot(expected_points,variables) + np.dot(expected_points,captaincy_variables)

    # SOLVE OPTIMIZATION

    status = model.solve()
    print(f'Status: {LpStatus[model.status]}')
    #print(f'Mean total points per gameweek: {model.objective.value()}')

    players = [str(var) for var in model.variables() if var.value()==1]
    captain = [player for player in players if 'cap' in player]
    captain = int(captain[0].replace('cap_', ''))
    players = [int(player) for player in players if 'cap' not in player]
    dream_team = df.loc[players]
    cost = (dream_team['price']).sum()
    exp_points = dream_team[expected_column].sum() + df.loc[captain, expected_column]
    print(f'Cost: {cost}')
    print(f'Expected points per week: {exp_points}')
    
    display(dream_team[['position','name', 'team', 'price',expected_column]].sort_values('position'))
    
    return players

# Data

In [5]:
#projections = pd.read_csv(f'../point_projections/xP_projections_week{gameweek}.csv', index_col=0)
projections = pd.read_csv(f'../point_projections/collective_xP_projections_week{gameweek}.csv', index_col=0)
projections = projections.rename(columns={'xP':'projected_points'})
projections.head(5)

,name,team,opp_team,was_home,gameweek,projected_points,std
0,Granit Xhaka,Arsenal,Manchester Utd,1,21,2.752840,0.122931
1,Granit Xhaka,Arsenal,Everton,0,22,3.174454,0.193868
2,Granit Xhaka,Arsenal,Brentford,1,23,2.880680,0.163509
3,Granit Xhaka,Arsenal,Manchester City,1,23,2.392095,0.126532
4,Granit Xhaka,Arsenal,Aston Villa,0,24,2.909361,0.165908


In [6]:
# decay for future projected points to model uncertainty increasing over time
decay_coeff = 0.875
projections['projected_points_with_decay'] = ( projections['projected_points']*decay_coeff
                                              **(projections['gameweek']-(gameweek+1)) )
projections.head(5)

,name,team,opp_team,was_home,gameweek,projected_points,std,projected_points_with_decay
0,Granit Xhaka,Arsenal,Manchester Utd,1,21,2.752840,0.122931,2.752840
1,Granit Xhaka,Arsenal,Everton,0,22,3.174454,0.193868,2.777648
2,Granit Xhaka,Arsenal,Brentford,1,23,2.880680,0.163509,2.205520
3,Granit Xhaka,Arsenal,Manchester City,1,23,2.392095,0.126532,1.831448
4,Granit Xhaka,Arsenal,Aston Villa,0,24,2.909361,0.165908,1.949045


In [7]:
fpl_df = pd.read_csv(f'../data/fpl_data/fpl_week_{gameweek}.csv', index_col=0)
fpl_df['name'] = fpl_df['name'].map(player_name_dict).fillna(fpl_df['name'])
fpl_df.head(5)

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,season,name,position,GW,value,selected,transfers_balance,opp_team_name,kickoff_time,was_home
0,100.0,100.0,58822,0,0,-3,3,0,2,0.5,...,2022-23,Cedric Alves Soares,DEF,20,4.2,11093.653,-78,Tottenham,2023-01-15T16:30:00Z,0
1,NaN,NaN,84450,0,0,0,0,1,3,3.5,...,2022-23,Granit Xhaka,MID,20,5.0,332809.590,-15602,Tottenham,2023-01-15T16:30:00Z,0
2,100.0,100.0,153256,0,0,-3,3,1,3,1.0,...,2022-23,Mohamed Elneny,MID,20,4.2,88749.224,-3499,Tottenham,2023-01-15T16:30:00Z,0
3,NaN,NaN,156074,0,0,-3,3,0,2,0.7,...,2022-23,Rob Holding,DEF,20,4.2,11093.653,-127,Tottenham,2023-01-15T16:30:00Z,0
4,100.0,100.0,167199,0,0,-2,2,0,3,2.7,...,2022-23,Thomas Partey,MID,20,4.8,55468.265,-443,Tottenham,2023-01-15T16:30:00Z,0


In [8]:
# choose time horizon for optimization
horizon = 10

# create dataframe to be given as input for optimization
df = projections[projections['gameweek']<=gameweek+horizon].groupby(by=['name']).mean()\
                                                        [['projected_points','projected_points_with_decay', 'std']] 
#df = df.to_frame()
df = df.reset_index()
df = df.rename(columns={'projected_points':'projected_points_per_game'})
#df['team'] = [fpl_df.loc[fpl_df['name']==name, 'team'].values[0] for name in df['name']]
df['price'] = [fpl_df.loc[fpl_df['name']==name, 'value'].values[0] for name in df['name']]
df['position'] = [fpl_df.loc[fpl_df['name']==name, 'element_type'].values[0] for name in df['name']]
position_dict={1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
df['position'] = df['position'].map(position_dict)
df['position'] = pd.Categorical(df['position'], ['GK','DEF','MID','FWD'])
df['team'] = [projections.loc[projections['name']==name, 'team'].head(1).values[0] for name in df['name']]
df['minutes'] = [fpl_df.loc[fpl_df['name']==name, 'minutes'].values[0] for name in df['name']]
# re-order columns
df = df[['name', 'team', 'position', 'price', 'minutes', 'projected_points_per_game', 'projected_points_with_decay', 'std']]
df.head(5)

,name,team,position,price,minutes,projected_points_per_game,projected_points_with_decay,std
0,Aaron Cresswell,West Ham,DEF,4.8,1533,2.958651,1.758182,0.091179
1,Aaron Hickey,Brentford,DEF,4.9,680,2.511695,1.485571,0.095925
2,Aaron Ramsdale,Arsenal,GK,4.9,1620,3.460496,2.076781,0.169442
3,Aaron Wan-Bissaka,Manchester Utd,DEF,4.3,432,3.073369,1.885017,0.133826
4,Abdoulaye Doucoure,Everton,MID,5.3,222,2.191134,1.289800,0.182997


In [9]:
# remove players who do not have enough minutes played
cutoff = 450
df = df[df['minutes']>=cutoff]

# edit some player prices according to my selling price
df.loc[df['name']=='Erling Haaland', 'price'] = 11.8
df.loc[df['name']=='Martinelli', 'price'] = 6.3

# Optimization

In [10]:
team_value = 101.7
money_in_the_bank = 1.5
allowed_bench_value = 17.5
max_price = team_value + money_in_the_bank - allowed_bench_value

## Without decay

In [ ]:
players = optimize(df, max_price, 'projected_points_per_game')

In [ ]:
ban = []
# ban some cheap keepers in good teams who don't actually play
cheap_keepers = ['Scott Carson', 'Zack Steffen', 'Jason Steele', 'Caoimhin Kelleher', 'Wayne Hennessey', 
                 'Karl Darlow', 'Darren Randolph']
cheap_keepers_ix = df[df['name'].isin(cheap_keepers)].index
ban = ban + list(cheap_keepers_ix)
# ban given players
ban_field = ['Marcos Alonso', 'Ben Chilwell', 'Riyad Mahrez', 'Darwin Nunez', 'Aymeric Laporte', 'Emil Krafth', 
             'Sergio Gomez', 'Nathaniel Phillips']
ban_field_ix = df[df['name'].isin(ban_field)].index
ban = ban + list(ban_field_ix)
my_df = df.drop(ban).copy()

In [ ]:
players = optimize(my_df, max_price, 'projected_points_per_game')

## With decay

In [ ]:
#players = optimize(df, max_price, 'projected_points_with_decay')

In [11]:
ban = []
# ban some cheap keepers in good teams who don't actually play
#cheap_keepers = ['Scott Carson', 'Zack Steffen', 'Jason Steele', 'Caoimhin Kelleher', 'Wayne Hennessey', 
#                 'Karl Darlow', 'Darren Randolph', 'Alphonse Areola', 'Asmir Begovic', 
#                'Sam Johnstone', 'Jack Butland']
#cheap_keepers_ix = df[df['name'].isin(cheap_keepers)].index
#ban = ban + list(cheap_keepers_ix)
# ban given players
#ban_field = ['Marcos Alonso', 'Riyad Mahrez', 'Nathaniel Phillips', 'Alexander Isak', 'Stefan Bajcetic',]
#ban_field_ix = df[df['name'].isin(ban_field)].index
#ban = ban + list(ban_field_ix)
my_df = df.drop(ban).copy()

In [12]:
players = optimize(my_df, max_price, 'projected_points_with_decay')

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/8b57c8fd93044204bb8db403699089b7-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/8b57c8fd93044204bb8db403699089b7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 322 COLUMNS
At line 4205 RHS
At line 4523 BOUNDS
At line 5100 ENDATA
Problem MODEL has 317 rows, 576 columns and 2154 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 35.2017 - 0.00 seconds
Cgl0004I processed model has 315 rows, 576 columns (576 integer (576 of which binary)) and 2016 elements
Cbc0038I Initial state - 4 integers unsatisfied sum - 0.879121
Cbc0038I Pass   1: suminf.    0.46512 (2) obj. -34.9333 iterations 8
Cbc0038I Solution found of -33.4561
Cbc0038I Before mini bran

,position,name,team,price,projected_points_with_decay
143,GK,Ederson,Manchester City,5.4,2.380077
256,DEF,John Stones,Manchester City,5.4,2.405395
28,DEF,Andrew Robertson,Liverpool,6.8,2.421485
286,DEF,Kieran Trippier,Newcastle Utd,6.0,2.421362
281,MID,Kevin De Bruyne,Manchester City,12.5,3.336867
343,MID,Martin degaard,Arsenal,6.8,3.131092
362,MID,Miguel Almiron,Newcastle Utd,5.9,2.665441
366,MID,Mohamed Salah,Liverpool,12.8,3.716213
471,MID,Solly March,Brighton,5.1,2.639432
14,FWD,Aleksandar Mitrovic,Fulham,7.1,3.114120


# Sensitivity analysis

Here we add small random Gaussian noise to each players' projected points and run the optimization with these new estimates. We repeat the process n times and keep a list of how many times each player appeared in the optimal team.

In [13]:
ban = []
# ban some cheap keepers in good teams who don't actually play
#cheap_keepers = ['Scott Carson', 'Zack Steffen', 'Jason Steele', 'Caoimhin Kelleher', 'Wayne Hennessey', 
#                 'Karl Darlow', 'Darren Randolph', 'Alphonse Areola', 'Asmir Begovic', 
#                'Sam Johnstone', 'Jack Butland']
#cheap_keepers_ix = df[df['name'].isin(cheap_keepers)].index
#ban = ban + list(cheap_keepers_ix)
# ban given players
#ban_field = ['Marcos Alonso', 'Riyad Mahrez', 'Nathaniel Phillips', 'Alexander Isak', 'Stefan Bajcetic']
#ban_field_ix = df[df['name'].isin(ban_field)].index
#ban = ban + list(ban_field_ix)
my_df = df.drop(ban).copy()

In [14]:
metric = 'projected_points_with_decay'
# set the standard deviation of the Gaussian noise to be a fraction of each players' projected points
#sigma_factor = 0.1
simulation_rounds = 100
results = []
for i in range(0,simulation_rounds):
    df_with_noise = my_df.copy()
    #df_with_noise[metric] = df_with_noise[metric] + np.random.normal(0, df_with_noise[metric]*sigma_factor)
    df_with_noise[metric] = df_with_noise[metric] + np.random.normal(0, df_with_noise['std'])
    players = optimize(df_with_noise, max_price, metric)
    results.append(players)

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/11d8d28979504964a3bd69ebdafb8f25-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/11d8d28979504964a3bd69ebdafb8f25-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 322 COLUMNS
At line 4205 RHS
At line 4523 BOUNDS
At line 5100 ENDATA
Problem MODEL has 317 rows, 576 columns and 2154 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 37.6927 - 0.00 seconds
Cgl0004I processed model has 315 rows, 576 columns (576 integer (576 of which binary)) and 2016 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.73913
Cbc0038I Solution found of -37.5705
Cbc0038I Before mini branch and bound, 574 integers at bound fixed and 0 continuous
Cbc0038I Fu

,position,name,team,price,projected_points_with_decay
114,GK,David Raya,Brentford,4.6,2.369398
256,DEF,John Stones,Manchester City,5.4,2.463625
328,DEF,Manuel Akanji,Manchester City,5.0,2.419801
502,DEF,Trent Alexander-Arnold,Liverpool,7.3,2.779336
314,MID,Luis Diaz,Liverpool,7.8,2.944058
36,MID,Antony,Manchester Utd,7.4,3.004567
366,MID,Mohamed Salah,Liverpool,12.8,4.009332
416,MID,Raheem Sterling,Chelsea,9.7,3.064261
471,MID,Solly March,Brighton,5.1,3.373152
14,FWD,Aleksandar Mitrovic,Fulham,7.1,2.710167


Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/99910933fc6a4026b761a26c49417c3b-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/99910933fc6a4026b761a26c49417c3b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 322 COLUMNS
At line 4205 RHS
At line 4523 BOUNDS
At line 5100 ENDATA
Problem MODEL has 317 rows, 576 columns and 2154 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 39.1149 - 0.00 seconds
Cgl0004I processed model has 315 rows, 576 columns (576 integer (576 of which binary)) and 2016 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.222222
Cbc0038I Pass   1: suminf.    0.10256 (2) obj. -39.0474 iterations 10
Cbc0038I Solution found of -37.2448
Cbc0038I Before mini bra

,position,name,team,price,projected_points_with_decay
143,GK,Ederson,Manchester City,5.4,2.510204
28,DEF,Andrew Robertson,Liverpool,6.8,2.741606
286,DEF,Kieran Trippier,Newcastle Utd,6.0,2.554972
502,DEF,Trent Alexander-Arnold,Liverpool,7.3,2.813909
281,MID,Kevin De Bruyne,Manchester City,12.5,3.503653
343,MID,Martin degaard,Arsenal,6.8,3.219539
362,MID,Miguel Almiron,Newcastle Utd,5.9,2.918055
366,MID,Mohamed Salah,Liverpool,12.8,4.180903
429,MID,Riyad Mahrez,Manchester City,7.5,3.644311
14,FWD,Aleksandar Mitrovic,Fulham,7.1,3.488517


Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/ad6ab84f189042238035af73ff705c59-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/ad6ab84f189042238035af73ff705c59-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 322 COLUMNS
At line 4205 RHS
At line 4523 BOUNDS
At line 5100 ENDATA
Problem MODEL has 317 rows, 576 columns and 2154 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 36.92 - 0.00 seconds
Cgl0004I processed model has 315 rows, 576 columns (576 integer (576 of which binary)) and 2016 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.75
Cbc0038I Pass   1: suminf.    0.13483 (2) obj. -36.8581 iterations 7
Cbc0038I Solution found of -35.2051
Cbc0038I Before mini branch and

...

,position,name,team,price,projected_points_with_decay
21,GK,Alisson,Liverpool,5.5,2.527538
162,DEF,Fabian Schar,Newcastle Utd,5.0,2.428476
256,DEF,John Stones,Manchester City,5.4,2.500044
28,DEF,Andrew Robertson,Liverpool,6.8,2.558876
343,MID,Martin degaard,Arsenal,6.8,3.616402
366,MID,Mohamed Salah,Liverpool,12.8,4.258401
416,MID,Raheem Sterling,Chelsea,9.7,2.920727
450,MID,Ryan Yates,Nottingham Forest,5.0,3.623766
14,FWD,Aleksandar Mitrovic,Fulham,7.1,2.914624
179,FWD,Harry Kane,Tottenham,11.7,2.966099


Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/9a54cb4d283747b1a1edb7f51e7a7e91-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/9a54cb4d283747b1a1edb7f51e7a7e91-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 322 COLUMNS
At line 4205 RHS
At line 4523 BOUNDS
At line 5100 ENDATA
Problem MODEL has 317 rows, 576 columns and 2154 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 36.5267 - 0.00 seconds
Cgl0004I processed model has 315 rows, 576 columns (576 integer (576 of which binary)) and 2016 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.4
Cbc0038I Solution found of -36.5035
Cbc0038I Before mini branch and bound, 574 integers at bound fixed and 0 continuous
Cbc0038I Full p

,position,name,team,price,projected_points_with_decay
143,GK,Ederson,Manchester City,5.4,2.601086
162,DEF,Fabian Schar,Newcastle Utd,5.0,2.252781
286,DEF,Kieran Trippier,Newcastle Utd,6.0,2.413776
422,DEF,Reece James,Chelsea,5.7,2.395028
281,MID,Kevin De Bruyne,Manchester City,12.5,3.351426
343,MID,Martin degaard,Arsenal,6.8,3.730524
429,MID,Riyad Mahrez,Manchester City,7.5,3.136219
67,MID,Bukayo Saka,Arsenal,8.1,3.185968
113,FWD,Darwin Nunez,Liverpool,8.8,3.187362
168,FWD,Gabriel Jesus,Arsenal,7.9,3.160311


Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/e78e52fbba0e4e4ca7f831d1aec76cc3-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/e78e52fbba0e4e4ca7f831d1aec76cc3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 322 COLUMNS
At line 4205 RHS
At line 4523 BOUNDS
At line 5100 ENDATA
Problem MODEL has 317 rows, 576 columns and 2154 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 36.8325 - 0.00 seconds
Cgl0004I processed model has 315 rows, 576 columns (576 integer (576 of which binary)) and 2016 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.137931
Cbc0038I Solution found of -36.831
Cbc0038I Before mini branch and bound, 574 integers at bound fixed and 0 continuous
Cbc0038I Fu

,position,name,team,price,projected_points_with_decay
21,GK,Alisson,Liverpool,5.5,2.537073
256,DEF,John Stones,Manchester City,5.4,2.357600
28,DEF,Andrew Robertson,Liverpool,6.8,2.407593
286,DEF,Kieran Trippier,Newcastle Utd,6.0,2.758313
272,MID,Kaoru Mitoma,Brighton,5.0,2.942449
281,MID,Kevin De Bruyne,Manchester City,12.5,3.069856
343,MID,Martin degaard,Arsenal,6.8,3.402283
362,MID,Miguel Almiron,Newcastle Utd,5.9,2.955385
366,MID,Mohamed Salah,Liverpool,12.8,3.537989
14,FWD,Aleksandar Mitrovic,Fulham,7.1,3.541943


In [15]:
my_df['dream_team_appearances'] = [sum(team.count(player_id) for team in results) for player_id in my_df.index]
my_df.head(5)

,name,team,position,price,minutes,projected_points_per_game,projected_points_with_decay,std,dream_team_appearances
0,Aaron Cresswell,West Ham,DEF,4.8,1533,2.958651,1.758182,0.091179,0
1,Aaron Hickey,Brentford,DEF,4.9,680,2.511695,1.485571,0.095925,0
2,Aaron Ramsdale,Arsenal,GK,4.9,1620,3.460496,2.076781,0.169442,10
5,Adam Armstrong,Southampton,FWD,5.5,929,2.347727,1.411195,0.160564,0
7,Adam Lallana,Brighton,MID,4.9,688,2.476469,1.485709,0.100678,0


In [16]:
print('Most appearances, all players:')
display(my_df.sort_values(by='dream_team_appearances', ascending=False).head(20))

Most appearances, all players:


,name,team,position,price,minutes,projected_points_per_game,projected_points_with_decay,std,dream_team_appearances
366,Mohamed Salah,Liverpool,MID,12.8,1597,6.249072,3.716213,0.554120,78
14,Aleksandar Mitrovic,Fulham,FWD,7.1,1373,5.324022,3.114120,0.627891,67
343,Martin degaard,Arsenal,MID,6.8,1432,5.211353,3.131092,0.390448,65
179,Harry Kane,Tottenham,FWD,11.7,1786,5.514803,3.214983,0.234590,59
281,Kevin De Bruyne,Manchester City,MID,12.5,1490,5.487887,3.336867,0.720107,53
286,Kieran Trippier,Newcastle Utd,DEF,6.0,1684,4.063698,2.421362,0.356078,51
256,John Stones,Manchester City,DEF,5.4,1090,3.959276,2.405395,0.164704,46
502,Trent Alexander-Arnold,Liverpool,DEF,7.3,1331,4.130870,2.478368,0.179629,43
28,Andrew Robertson,Liverpool,DEF,6.8,1145,4.031555,2.421485,0.189691,41
143,Ederson,Manchester City,GK,5.4,1710,3.914143,2.380077,0.206418,35


In [17]:
print('GOALKEEPERS')
display(my_df[my_df['position']=='GK'].sort_values(by='dream_team_appearances', ascending=False).head(10))

GOALKEEPERS


,name,team,position,price,minutes,projected_points_per_game,projected_points_with_decay,std,dream_team_appearances
143,Ederson,Manchester City,GK,5.4,1710,3.914143,2.380077,0.206418,35
21,Alisson,Liverpool,GK,5.5,1620,3.637754,2.187742,0.213097,21
114,David Raya,Brentford,GK,4.6,1710,3.530171,2.094661,0.147030,12
280,Kepa Arrizabalaga,Chelsea,GK,4.6,945,3.506402,2.019539,0.220929,11
2,Aaron Ramsdale,Arsenal,GK,4.9,1620,3.460496,2.076781,0.169442,10
115,David de Gea,Manchester Utd,GK,4.9,1710,3.315255,2.037853,0.212398,9
390,Nick Pope,Newcastle Utd,GK,5.4,1710,3.286577,1.972295,0.130859,1
117,Dean Henderson,Nottingham Forest,GK,4.7,1620,3.281747,1.945289,0.147996,1
261,Jordan Pickford,Everton,GK,4.4,1620,2.870057,1.687913,0.159035,0
431,Robert Sanchez,Brighton,GK,4.6,1620,3.144740,1.900403,0.114007,0


In [18]:
print('DEFENDERS')
display(my_df[my_df['position']=='DEF'].sort_values(by='dream_team_appearances', ascending=False).head(20))

DEFENDERS


,name,team,position,price,minutes,projected_points_per_game,projected_points_with_decay,std,dream_team_appearances
286,Kieran Trippier,Newcastle Utd,DEF,6.0,1684,4.063698,2.421362,0.356078,51
256,John Stones,Manchester City,DEF,5.4,1090,3.959276,2.405395,0.164704,46
502,Trent Alexander-Arnold,Liverpool,DEF,7.3,1331,4.130870,2.478368,0.179629,43
28,Andrew Robertson,Liverpool,DEF,6.8,1145,4.031555,2.421485,0.189691,41
293,Kyle Walker,Manchester City,DEF,4.8,702,3.844022,2.334047,0.156548,34
422,Reece James,Chelsea,DEF,5.7,610,3.747718,2.163886,0.168809,15
167,Gabriel Dos Santos,Arsenal,DEF,5.2,1620,3.707887,2.223842,0.154962,15
240,Joao Cancelo,Manchester City,DEF,7.2,1273,3.680406,2.234316,0.263473,11
328,Manuel Akanji,Manchester City,DEF,5.0,1106,3.641908,2.211414,0.143788,9
162,Fabian Schar,Newcastle Utd,DEF,5.0,1604,3.507601,2.096057,0.165773,9


In [19]:
print('MIDFIELDERS')
display(my_df[my_df['position']=='MID'].sort_values(by='dream_team_appearances', ascending=False).head(20))

MIDFIELDERS


,name,team,position,price,minutes,projected_points_per_game,projected_points_with_decay,std,dream_team_appearances
366,Mohamed Salah,Liverpool,MID,12.8,1597,6.249072,3.716213,0.554120,78
343,Martin degaard,Arsenal,MID,6.8,1432,5.211353,3.131092,0.390448,65
281,Kevin De Bruyne,Manchester City,MID,12.5,1490,5.487887,3.336867,0.720107,53
429,Riyad Mahrez,Manchester City,MID,7.5,755,4.688844,2.847855,0.428272,34
362,Miguel Almiron,Newcastle Utd,MID,5.9,1571,4.490196,2.665441,0.308753,32
416,Raheem Sterling,Chelsea,MID,9.7,1111,4.896904,2.837224,0.506632,31
471,Solly March,Brighton,MID,5.1,1542,4.396456,2.639432,0.329541,29
67,Bukayo Saka,Arsenal,MID,8.1,1534,4.648694,2.783722,0.306544,28
450,Ryan Yates,Nottingham Forest,MID,5.0,1347,2.820723,1.665576,0.982367,17
36,Antony,Manchester Utd,MID,7.4,760,4.291511,2.634993,0.283583,16


In [20]:
print('FORWARDS')
display(my_df[my_df['position']=='FWD'].sort_values(by='dream_team_appearances', ascending=False).head(20))

FORWARDS


,name,team,position,price,minutes,projected_points_per_game,projected_points_with_decay,std,dream_team_appearances
14,Aleksandar Mitrovic,Fulham,FWD,7.1,1373,5.324022,3.114120,0.627891,67
179,Harry Kane,Tottenham,FWD,11.7,1786,5.514803,3.214983,0.234590,59
199,Ivan Toney,Brentford,FWD,7.6,1521,4.761145,2.826816,0.299998,33
168,Gabriel Jesus,Arsenal,FWD,7.9,1220,4.608076,2.757934,0.256718,31
156,Erling Haaland,Manchester City,FWD,11.8,1485,4.995297,3.029246,0.809104,30
113,Darwin Nunez,Liverpool,FWD,8.8,868,4.225394,2.529142,0.605963,23
269,Kai Havertz,Chelsea,FWD,7.8,1329,3.842484,2.220200,0.182317,1
396,Ollie Watkins,Aston Villa,FWD,7.1,1448,3.751370,2.197748,0.130589,0
393,Odsonne Edouard,Crystal Palace,FWD,5.2,887,2.429036,1.429317,0.113945,0
386,Neal Maupay,Everton,FWD,6.0,677,2.683842,1.582858,0.397685,0
